In [1]:
# Install PyTorch Geometric & dependensinya untuk Colab
!pip install torch==2.0.0+cu118 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.0.0+cu118.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-2.0.0+cu118.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-2.0.0+cu118.html
!pip install torch-spline-conv -f https://data.pyg.org/whl/torch-2.0.0+cu118.html
!pip install torch-geometric
!pip install pandas scikit-learn matplotlib
# Downgrade NumPy ke versi stabil 1.x untuk kompatibilitas
!pip install numpy==1.24.4



Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 15.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 93.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 92.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 93.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 91.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 91.4 MB/s eta 0:00:00
INFO: pip is still looking at multiple versions of torchvision to determine which version is compatible with other re

In [1]:
import pandas as pd
import torch
import torch.nn.functional as F
from torch_geometric.data import Dataset, Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
import ast


In [2]:
class CSVGraphDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.df = pd.read_csv(csv_file)
        self.df["Node_Features"] = self.df["Node_Features"].apply(ast.literal_eval)
        self.df["Edge_Index"] = self.df["Edge_Index"].apply(ast.literal_eval)
        self.label_map = {"tidak_estetik": 0, "estetik": 1}
        self.transform = transform
        super().__init__()

    def len(self):
        return len(self.df)

    def get(self, idx):
        row = self.df.iloc[idx]
        x = torch.tensor(row["Node_Features"], dtype=torch.float)
        edge_index = torch.tensor(row["Edge_Index"], dtype=torch.long)
        if edge_index.shape[0] != 2:
            edge_index = edge_index.t().contiguous()
        y = torch.tensor(self.label_map[row["Category"]], dtype=torch.long)
        data = Data(x=x, edge_index=edge_index, y=y)
        data.file_name = row["Filename"]
        return self.transform(data) if self.transform else data


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/Hasil_Ekstraksi/graph_dataset_fresh.csv")
print(df.columns)
print(df.head(1)["Node_Features"].values[0])


ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Hasil_Ekstraksi/graph_dataset_fresh.csv")

# Cek baris yang tidak bisa dieval
import ast

def is_valid_literal(x):
    try:
        ast.literal_eval(x)
        return True
    except:
        return False

invalid_rows = df[~df["Node_Features"].apply(is_valid_literal)]
print("Jumlah baris bermasalah:", len(invalid_rows))
print(invalid_rows[["Filename", "Node_Features"]].head())


In [ ]:
print(invalid_rows["Filename"].tolist())


In [ ]:
import pandas as pd

# Path file CSV lama
csv_path = "/content/drive/MyDrive/Hasil_Ekstraksi/graph_dataset_fresh.csv"

# Daftar filename yang ingin dihapus
hapus_list = [
    '18650.jpg', '22236.jpg', '1524.jpg', '2663.jpg', '3660.jpg',
    '4873.jpg', '5406.jpg', '5847.jpg', '6737.jpg', '9981.jpg'
]

# Baca CSV lama
df = pd.read_csv(csv_path)

# Filter baris yang tidak termasuk file yang akan dihapus
df_bersih = df[~df["Filename"].isin(hapus_list)]

# Simpan ulang ke file asli
df_bersih.to_csv(csv_path, index=False)

print(f"✅ File berhasil diperbarui dan disimpan kembali ke:\n{csv_path}")
print(f"📊 Total baris setelah dibersihkan: {len(df_bersih)}")


In [ ]:
dataset = CSVGraphDataset("/content/drive/MyDrive/Hasil_Ekstraksi/graph_dataset_fresh.csv")


train_loader = DataLoader(dataset, batch_size=10, shuffle=True)
print(f"Total graph: {len(dataset)}")



In [ ]:



sample = dataset[1881]
print("x shape:", sample.x.shape)
print("edge_index shape:", sample.edge_index.shape)
print("y:", sample.y)


In [ ]:
print(self.df.columns)  # akan tampil: Index(['Filename', 'Node_Features', ...])


In [ ]:
# Cek label dari semua graph
from collections import Counter

all_labels = [dataset[i].y.item() for i in range(len(dataset))]
print("🔢 Jumlah label per kelas:")
print(Counter(all_labels))

# Atau tampilkan semua:
print("📋 Semua label di dataset:")
print(all_labels)


In [ ]:
class GCNGraphClassifier(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.lin = torch.nn.Linear(hidden_channels, out_channels)

    def forward(self, x, edge_index, batch=None):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        if batch is not None and x.size(0) > batch.max().item():
            x = global_mean_pool(x, batch)
            print("masuk")
        else:
            x = x.mean(dim=0, keepdim=True)
        return self.lin(x)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool

class GCNGraphClassifier(nn.Module):
    def __init__(self, input_dim=2800, hidden_dim=128, num_classes=2):
        super(GCNGraphClassifier, self).__init__()

        self.gcn1 = GCNConv(input_dim, hidden_dim)
        self.gcn2 = GCNConv(hidden_dim, hidden_dim)
        self.dropout = nn.Dropout(p=0.3)
        self.classifier = nn.Linear(hidden_dim, num_classes)

    def forward(self, x, edge_index, batch):
        x = self.gcn1(x, edge_index)
        x = F.relu(x)
        x = self.dropout(x)

        x = self.gcn2(x, edge_index)
        x = F.relu(x)

        x = global_mean_pool(x, batch)  # agregasi semua node dalam graf
        out = self.classifier(x)
        return out


In [ ]:
# Pastikan class GCNGraphClassifier sudah didefinisikan sebelum ini

# Ambil 1 sampel dari dataset untuk inisialisasi model
sample = dataset[0]

# Inisialisasi model
model = GCNGraphClassifier(sample.num_node_features, 64, 2).to(device)

# Optimizer dan loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()


In [ ]:
for epoch in range(1, 101):
    model.train()
    total_loss = 0

    for batch_idx, batch in enumerate(train_loader):
        batch = batch.to(device)
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index, batch.batch)
        loss = criterion(out, batch.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        # 🔎 Tambahan: tampilkan output GCN per graf
        out_cpu = out.cpu()
        preds = out_cpu.argmax(dim=1)
        labels = batch.y.cpu()

        # Ambil nama file kalau tersedia
        try:
            data_list = batch.to_data_list()
            file_names = [d.file_name if hasattr(d, 'file_name') else f"G_{i}" for i, d in enumerate(data_list)]
        except:
            file_names = [f"G_{i}" for i in range(len(labels))]

        print(f"\n[Epoch {epoch} | Batch {batch_idx+1}]")
        for i in range(len(preds)):
            print(f"Graf ke-{i+1} ({file_names[i]}):")
            print(f"  Logit     : {out_cpu[i].tolist()}")
            print(f"  Prediksi  : {preds[i].item()}")
            print(f"  Label Asli: {labels[i].item()}")

    avg_loss = total_loss / len(train_loader)
    print(f"\n[Epoch {epoch}] Train Loss: {avg_loss:.4f}")


In [ ]:
import torch
from torch_geometric.loader import DataLoader
from torch.utils.data import random_split
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Setup device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Split dataset 80% train, 20% test
total = len(dataset)
train_size = int(0.8 * total)
test_size = total - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

# Inisialisasi model
sample = dataset[0]
model = GCNGraphClassifier(sample.num_node_features, 64, 2).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

train_losses = []
test_losses = []
eval_epochs = []
accs = []
precisions = []
recalls = []
f1s = []

# Training
for epoch in range(1, 101):
    model.train()
    total_train_loss = 0

    for batch in train_loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index, batch.batch)
        loss = criterion(out, batch.y)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()

    avg_train_loss = total_train_loss / len(train_loader)

    # Hitung test loss
    model.eval()
    total_test_loss = 0
    with torch.no_grad():
        for batch in test_loader:
            batch = batch.to(device)
            out = model(batch.x, batch.edge_index, batch.batch)
            loss = criterion(out, batch.y)
            total_test_loss += loss.item()
    avg_test_loss = total_test_loss / len(test_loader)

    # Simpan loss setiap 10 epoch
    if epoch % 10 == 0:
        train_losses.append(avg_train_loss)
        test_losses.append(avg_test_loss)

    # Evaluasi setiap 5 epoch
    if epoch % 5 == 0:
        model.eval()
        y_true = []
        y_pred = []
        misclassified_files = []

        with torch.no_grad():
            for batch in test_loader:
                batch = batch.to(device)
                out = model(batch.x, batch.edge_index, batch.batch)
                preds = out.argmax(dim=1).cpu()
                labels = batch.y.cpu()

                y_true.extend(labels.numpy())
                y_pred.extend(preds.numpy())

                # ⬅️ Tracking file yang salah prediksi
                if hasattr(batch, 'file_name'):  # cek apakah file_name tersedia
                    for i in range(len(preds)):
                        if preds[i] != labels[i]:
                            misclassified_files.append(batch.file_name[i])


        acc = accuracy_score(y_true, y_pred)
        prec = precision_score(y_true, y_pred, average='weighted', zero_division=0)
        rec = recall_score(y_true, y_pred, average='weighted', zero_division=0)
        f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)

        accs.append(acc)
        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)
        eval_epochs.append(epoch)

        print(f"[Epoch {epoch}] Train Loss: {avg_train_loss:.4f} | Test Loss: {avg_test_loss:.4f}")
        print(f"           Acc: {acc:.4f} | Prec: {prec:.4f} | Recall: {rec:.4f} | F1: {f1:.4f}")
        if misclassified_files:
            print(f"  Salah prediksi ({len(misclassified_files)} graf):")
            for name in misclassified_files[:5]:  # tampilkan 5 contoh saja
                print(f"   - {name}")


# Plot loss
plt.figure(figsize=(10, 5))
plt.plot(range(10, 101, 10), train_losses, label='Train Loss', marker='o')
plt.plot(range(10, 101, 10), test_losses, label='Test Loss', marker='s')
plt.title('Loss per 10 Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Plot metrik evaluasi
plt.figure(figsize=(10, 6))
plt.plot(eval_epochs, accs, label='Accuracy', marker='o')
plt.plot(eval_epochs, precisions, label='Precision', marker='s')
plt.plot(eval_epochs, recalls, label='Recall', marker='^')
plt.plot(eval_epochs, f1s, label='F1-score', marker='d')
plt.title('Evaluasi Model Setiap 5 Epoch')
plt.xlabel('Epoch')
plt.ylabel('Score')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
out = model(batch.x, batch.edge_index, batch.batch)  # output mentah
probs = torch.softmax(out, dim=1)                    # ubah jadi probabilitas

# Ambil probabilitas kelas estetik (label = 1)
prob_estetik = probs[:, 1]  # shape [batch_size]

for i, prob in enumerate(prob_estetik):
    print(f"Graf ke-{i+1} → Estetik dengan kemungkinan {prob.item():.2%}")
